In [1]:
from datasets import load_dataset
import pandas as pd

sample_size = 100000  # you can reduce this (e.g., 10000) if low on RAM/storage

# Load dataset in streaming mode to avoid full download
dataset = load_dataset("ai4bharat/wiki-translate", split="train", streaming=True)

# Extract only English ↔ Hindi pairs
data = []
for i, row in enumerate(dataset):
    if "pan_Guru" in row:
        data.append({
            "Punjabi": row["pan_Guru"]
        })
        if i + 1 >= sample_size:
            break

# Convert to DataFrame
df = pd.DataFrame(data)
print("✅ Loaded", len(df), "Punjabi Datset : ")

# Save as CSV for preprocessing/training
df.to_csv("wiki_translate_punjabi_dataset.csv", index=False, encoding="utf-8")
print("💾 Saved file: wiki_translate_punjabi_dataset.csv")


Repo card metadata block was not found. Setting CardData to empty.


Resolving data files:   0%|          | 0/256 [00:00<?, ?it/s]

'HTTPSConnectionPool(host='cas-bridge.xethub.hf.co', port=443): Read timed out.' thrown while requesting GET https://huggingface.co/datasets/ai4bharat/wiki-translate/resolve/68badfb8db907f2c0ebda8480a08ccfc0313def4/train/wiki_train_0003_of_0255.parquet
Retrying in 1s [Retry 1/5].
'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 0255f416-da86-41d7-b468-1c855c855fc7)')' thrown while requesting GET https://huggingface.co/datasets/ai4bharat/wiki-translate/resolve/68badfb8db907f2c0ebda8480a08ccfc0313def4/train/wiki_train_0003_of_0255.parquet
Retrying in 2s [Retry 2/5].
'(MaxRetryError('HTTPSConnectionPool(host=\'huggingface.co\', port=443): Max retries exceeded with url: /datasets/ai4bharat/wiki-translate/resolve/68badfb8db907f2c0ebda8480a08ccfc0313def4/train/wiki_train_0003_of_0255.parquet (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x00000205145D8410>: Failed to resolve \'huggingf

✅ Loaded 100000 Punjabi Datset : 
💾 Saved file: wiki_translate_punjabi_dataset.csv


In [3]:
import pandas as pd

# Load your saved subset
df = pd.read_csv("wiki_translate_punjabi_dataset.csv")

print("✅ Dataset loaded successfully!")
print(f"Total rows: {len(df)}")
df.head()

✅ Dataset loaded successfully!
Total rows: 100000


,Punjabi
0,"\nਵੱਡਾ (ਬੇਓਨਸੇ ਗੀਤ)\n\n""ਵੱਡਾ"" (ਵੱਡੇ ਅੱਖਰ ਵਿੱਚ ..."
1,ਕੋਰੀਆਈ. ਸ਼ਬਦ-ਵਿਗਿਆਨ 2. ਕ੍ਰਿਆ। ਸੰਯੋਜਨ. ਨੋਟਃ ਹਾਲ...
2,\nਹੈਰੀਏਟ ਚਿੱਟਾ ਮਛੇਰਾ\n\nਹੈਰੀਏਟ ਵ੍ਹਾਈਟ ਫਿਸ਼ਰ ਐਂ...
3,\nਆਈਰਿਸ (ਮਿਥਿਹਾਸ)\n\nਪ੍ਰਾਚੀਨ ਯੂਨਾਨੀ ਧਰਮ ਅਤੇ ਮਿ...
4,\nਆਇਓਨਿਸ ਕੋਂਟੋਇਆਨਿਸ\n\nਇਓਨਿਸ ਕੋਂਟੋਇਆਨਿਸ (ਜਨਮ ਜ...


In [4]:
print("🔎 Checking for null values:")
print(df.isnull().sum())

🔎 Checking for null values:
Punjabi    0
dtype: int64


In [5]:
print(df.shape)

(100000, 1)


In [1]:
import pandas as pd
import re
import unicodedata
from pathlib import Path

class PunjabiDataCleaner:
    """
    Comprehensive data cleaner for Punjabi Wikipedia text data.
    Handles common cleaning tasks for Indic language text.
    """
    
    def __init__(self):
        # Punjabi Unicode range: \u0A00-\u0A7F
        self.punjabi_pattern = re.compile(r'[\u0A00-\u0A7F]')
        
    def remove_urls(self, text):
        """Remove URLs from text"""
        url_pattern = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
        return re.sub(url_pattern, '', text)
    
    def remove_email(self, text):
        """Remove email addresses"""
        email_pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
        return re.sub(email_pattern, '', text)
    
    def remove_html_tags(self, text):
        """Remove HTML tags"""
        html_pattern = re.compile(r'<.*?>')
        return html_pattern.sub('', text)
    
    def remove_special_chars(self, text):
        """Remove special characters but keep Punjabi characters and basic punctuation"""
        # Keep Punjabi chars, spaces, common punctuation
        pattern = r'[^\u0A00-\u0A7F\s।॥,.!?;:()\-\'\"0-9a-zA-Z]'
        return re.sub(pattern, ' ', text)
    
    def normalize_whitespace(self, text):
        """Normalize multiple spaces/tabs/newlines to single space"""
        text = re.sub(r'\s+', ' ', text)
        return text.strip()
    
    def remove_extra_punctuation(self, text):
        """Remove repeated punctuation marks"""
        text = re.sub(r'([।॥,.!?;:])\1+', r'\1', text)
        return text
    
    def normalize_unicode(self, text):
        """Normalize Unicode characters (NFC normalization)"""
        return unicodedata.normalize('NFC', text)
    
    def remove_digits_only_lines(self, text):
        """Remove lines that contain only digits"""
        if re.match(r'^\d+$', text.strip()):
            return ''
        return text
    
    def has_punjabi_content(self, text):
        """Check if text contains Punjabi characters"""
        return bool(self.punjabi_pattern.search(text))
    
    def clean_text(self, text):
        """
        Apply all cleaning steps to a single text string
        """
        if pd.isna(text) or text == '':
            return ''
        
        # Convert to string
        text = str(text)
        
        # Apply cleaning steps in order
        text = self.remove_urls(text)
        text = self.remove_email(text)
        text = self.remove_html_tags(text)
        text = self.normalize_unicode(text)
        text = self.remove_special_chars(text)
        text = self.remove_extra_punctuation(text)
        text = self.normalize_whitespace(text)
        text = self.remove_digits_only_lines(text)
        
        # Keep only if it has Punjabi content and is not too short
        if self.has_punjabi_content(text) and len(text.strip()) > 10:
            return text
        
        return ''


def load_and_clean_csv(csv_path, column_name=None, sample_size=None):
    """
    Load CSV file and clean the data
    
    Args:
        csv_path: Path to CSV file
        column_name: Name of the column containing text (if None, uses first column)
        sample_size: Number of rows to sample (if None, uses all rows)
    """
    print(f"Loading CSV from: {csv_path}")
    df = pd.read_csv(csv_path)
    
    print(f"Original dataset shape: {df.shape}")
    print(f"Columns: {df.columns.tolist()}")
    
    # Use specified column or first column
    if column_name is None:
        column_name = df.columns[0]
    
    print(f"Using column: '{column_name}'")
    
    # Sample if needed
    if sample_size and sample_size < len(df):
        df = df.sample(n=sample_size, random_state=42)
        print(f"Sampled {sample_size} rows")
    
    # Initialize cleaner
    cleaner = PunjabiDataCleaner()
    
    # Clean the data
    print("\nCleaning data...")
    df['cleaned_text'] = df[column_name].apply(cleaner.clean_text)
    
    # Remove empty strings
    df = df[df['cleaned_text'] != '']
    
    print(f"After cleaning: {len(df)} rows remaining")
    
    return df['cleaned_text'].tolist()


def save_to_txt(texts, output_path):
    """
    Save cleaned texts to a .txt file (one sentence per line)
    """
    print(f"\nSaving cleaned data to: {output_path}")
    
    with open(output_path, 'w', encoding='utf-8') as f:
        for text in texts:
            # Split into sentences if needed (by Punjabi danda)
            sentences = re.split(r'[।॥]', text)
            for sent in sentences:
                sent = sent.strip()
                if sent and len(sent) > 5:  # Only save non-empty sentences
                    f.write(sent + '\n')
    
    print(f"Saved {len(texts)} texts to {output_path}")


if __name__ == "__main__":
    
    # Configuration - UPDATE THESE PATHS!
    # Example 1: File in same directory
    CSV_FILE = "wiki_translate_punjabi_dataset.csv"  # Change to your CSV filename
    
    # Example 2: Full path (Windows)
    # CSV_FILE = r"C:\Users\YourName\Downloads\wiki_translate_punjabi.csv"
    
    # Example 3: Full path (Linux/Mac)
    # CSV_FILE = "/home/username/Downloads/wiki_translate_punjabi.csv"
    
    COLUMN_NAME = None  # Set to None to use first column, or specify like "text" or "content"
    OUTPUT_TXT = "punjabi_cleaned.txt"  # Output file path
    
    # Optional: Use a sample for testing (set to None to use all data)
    SAMPLE_SIZE = None  # e.g., 10000 for testing, None for full dataset
    
    print("="*70)
    print("PUNJABI DATASET CLEANING PIPELINE")
    print("="*70)
    
    # Step 1: Load and clean the data
    try:
        cleaned_texts = load_and_clean_csv(
            csv_path=CSV_FILE,
            column_name=COLUMN_NAME,
            sample_size=SAMPLE_SIZE
        )
        
        # Display sample
        print("\n--- Sample Cleaned Text ---")
        for i, text in enumerate(cleaned_texts[:3], 1):
            print(f"{i}. {text[:200]}...")
        
    except FileNotFoundError:
        print(f"Error: File '{CSV_FILE}' not found!")
        print("Please make sure the CSV file is in the same directory.")
        exit(1)
    except Exception as e:
        print(f"Error loading data: {e}")
        exit(1)
    
    # Step 2: Save to text file
    try:
        save_to_txt(cleaned_texts, OUTPUT_TXT)
    except Exception as e:
        print(f"Error saving to text file: {e}")
        exit(1)
    
    print("\n" + "="*70)
    print("CLEANING COMPLETED SUCCESSFULLY!")
    print("="*70)
    print(f"\nOutput file:")
    print(f"  - Cleaned text: {OUTPUT_TXT}")
    print(f"  - Total lines saved: {sum(1 for _ in open(OUTPUT_TXT, encoding='utf-8'))}")
    print("\nYou can now use this file for tokenization!")

PUNJABI DATASET CLEANING PIPELINE
Loading CSV from: wiki_translate_punjabi_dataset.csv
Original dataset shape: (100000, 1)
Columns: ['Punjabi']
Using column: 'Punjabi'

Cleaning data...
After cleaning: 99973 rows remaining

--- Sample Cleaned Text ---
1. ਵੱਡਾ (ਬੇਓਨਸੇ ਗੀਤ) "ਵੱਡਾ" (ਵੱਡੇ ਅੱਖਰ ਵਿੱਚ ਸ਼ੈਲੀਬੱਧ) ਅਮਰੀਕੀ ਗਾਇਕ ਬਿਓਨਸੇ ਦੁਆਰਾ 2019 ਦੀ ਐਲਬਮ "" ਤੋਂ ਇੱਕ ਗੀਤ ਹੈ ਅਤੇ 2020 ਦੀ ਫਿਲਮ "" ਬਲੈਕ ਇਜ਼ ਕਿੰਗ "" ਵਿੱਚ ਪ੍ਰਦਰਸ਼ਿਤ ਕੀਤਾ ਗਿਆ ਹੈ। ਪਿਛੋਕਡ਼. ਇਹ ਗੀਤ ਬਿਓਨਸੇ ਦੇ ਸਟ...
2. ਕੋਰੀਆਈ. ਸ਼ਬਦ-ਵਿਗਿਆਨ 2. ਕ੍ਰਿਆ। ਸੰਯੋਜਨ. ਨੋਟਃ ਹਾਲਾਂਕਿ ਤਜਵੀਜ਼ ਅਨੁਸਾਰ ਸੰਭਵ ਹੈ, ਫਾਰਮ ਅਭਿਆਸ ਵਿੱਚ ਘੱਟ ਹੀ ਵਰਤੇ ਜਾਂਦੇ ਹਨ। (, ਆਦਿ) ਦੇ ਰੂਪ ਵਿੱਚ ਦਿੱਤੇ ਗਏ ਸਾਰੇ ਰੂਪਾਂ ਲਈ।) ਹੇਠਾਂ, ਨੋਟ ਕਰੋ ਕਿ ਆਮ ਰੂਪ (, ਆਦਿ) ਹੈ।)। ਸ਼ਬਦ...
3. ਹੈਰੀਏਟ ਚਿੱਟਾ ਮਛੇਰਾ ਹੈਰੀਏਟ ਵ੍ਹਾਈਟ ਫਿਸ਼ਰ ਐਂਡਰਿਊ (1861-1939) ਇੱਕ ਅਮਰੀਕੀ ਸੀ ਜੋ ਇੱਕ ਲੋਕੋਮੋਬਾਈਲ ਵਿੱਚ ਦੁਨੀਆ ਦਾ ਚੱਕਰ ਲਗਾਉਣ ਵਾਲੀ ਪਹਿਲੀ ਔਰਤ ਵਜੋਂ ਜਾਣੀ ਜਾਂਦੀ ਸੀ। ਜਨਮ ਅਤੇ ਮੁਢਲਾ ਜੀਵਨ। ਹੈਰੀਏਟ ਵ੍ਹਾਈਟ ਦਾ ਜਨਮ 31 ਮਾਰਚ 1...

Saving cleaned data to: punjabi_cleaned.txt
Saved 99973 texts to punjabi_cleaned.txt

CLEANING COMPLETED SUCCESSFULLY!

Output